In [ ]:
from apriltag_detection import *
import cv2
from PIDcontrols import *

vertical_pid = PID(30, 0, 0, 100)
horizontal_pid = PID(30, 0, 0, 100)

video = cv2.VideoCapture('AprilTagTest.mkv')
pose = detect_tag(video)
headingE = pose[0]
depthE = pose[1]

horizontal_output, vertical_output = PID_tags((1920, 1080), headingE, depthE, horizontal_pid, vertical_pid)


horizontal_output, vertical_output


vertical_pid = PID(30, 0, 0, 100)
horizontal_pid = PID(30, 0, 0, 100)

video = cv2.VideoCapture('AprilTagTest.mkv')

'''fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_file = 'april_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_file, fourcc, 30, (width, height))'''

ret, frame = video.read()
i = 0
cameraMatrix = numpy.array([ 1060.71, 0, 960, 0, 1060.71, 540, 0, 0, 1]).reshape((3,3))
camera_params = ( cameraMatrix[0,0], cameraMatrix[1,1], cameraMatrix[0,2], cameraMatrix[1,2] )
at_detector = Detector(families='tag36h11',
                    nthreads=1,
                    quad_decimate=1.0,
                    quad_sigma=0.0,
                    refine_edges=1,
                    decode_sharpening=0.25,
                    debug=0)

PID_horizontal_output_lists = []
PID_vertical_output_lists = []

while ret:
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    color_img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    tags = at_detector.detect(img, True, camera_params, tag_size = 0.1)
    for tag in tags:
        for idx in range(len(tag.corners)):
            cv2.line(color_img, tuple(tag.corners[idx - 1, :].astype(int)), tuple(tag.corners[idx, :].astype(int)), (0, 255, 0))
            cv2.circle(color_img, (int(tag.center[0].item()),int(tag.center[1].item())), 50, (0, 0, 255), 2)
        
        pos = tag.center
        horizontal_output, vertical_output = PID_tags(frame.shape, pos[0], pos[1], horizontal_pid, vertical_pid)
        PID_horizontal_output_lists.append(horizontal_output)
        PID_vertical_output_lists.append(vertical_output)
    ret, frame = video.read()